### 匯入套件

In [449]:
import pandas as pd
import numpy as np
import os


### 匯入檔案

In [450]:
current_dir = os.getcwd()
input_path=os.path.join(current_dir,r"../../data/104_cleaned_0718.csv")

df=pd.read_csv(input_path)
print(df.head())

       上架日期  申請人數        產業              公司名稱  \
0  20250709     5  雜誌／期刊出版業  今周刊_今周文化事業股份有限公司   
1  20250715     4    人力仲介代徵      藝珂人事顧問股份有限公司   
2  20250717     6    半導體製造業      錼創顯示科技股份有限公司   
3  20250604     5  大專校院教育事業          國立臺北科技大學   
4  20250625    11   工商顧問服務業          世銳顧問有限公司   

                                         description   縣市  \
0  1.【必備】Google Analytics技能。 \n2.【必備】熟稔Excel(函數、樞...  台北市   
1  數據洞察與分析\n分析商品分類、顧客行為、流量來源、轉換率等電商關鍵指標，找出成長機會與潛在...  台北市   
2  1. 分析生產數據與大數據建模，協助建立品質監測與異常偵測模型，提供優化建議。\n2. 參與...  苗栗縣   
3  作，成果應用於企業與政府智慧城市專案。加入我們，您將成為城市科學的先鋒，與世界領先的團隊共創...  台北市   
4  題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...  台北市   

                        職稱      工作編號  工作型態         經度  ...  是否遠距   薪資上限  \
0         數據分析師【今周刊─數位內容部】  14090732     2  25.051911  ...     0      0   
1              [電商] 數據分析主管  14739101     0  25.040652  ...     0      0   
2                  數據分析工程師  14535114     0  24.711336  ...     0      0   
3  MITxTaipei Tech 都市數據分析師  13817654  

In [451]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   上架日期         866 non-null    int64  
 1   申請人數         866 non-null    int64  
 2   產業           866 non-null    object 
 3   公司名稱         866 non-null    object 
 4   description  866 non-null    object 
 5   縣市           866 non-null    object 
 6   職稱           866 non-null    object 
 7   工作編號         866 non-null    int64  
 8   工作型態         866 non-null    int64  
 9   經度           866 non-null    float64
 10  緯度           866 non-null    float64
 11  major        866 non-null    object 
 12  學歷要求         866 non-null    object 
 13  是否遠距         866 non-null    int64  
 14  薪資上限         866 non-null    int64  
 15  薪資下限         866 non-null    int64  
 16  工作時段         258 non-null    object 
 17  職務分類代碼       866 non-null    object 
 18  是否積極徵才       866 non-null    int64  
 19  條件要求    

In [452]:
df.isnull().sum()

上架日期             0
申請人數             0
產業               0
公司名稱             0
description      0
縣市               0
職稱               0
工作編號             0
工作型態             0
經度               0
緯度               0
major            0
學歷要求             0
是否遠距             0
薪資上限             0
薪資下限             0
工作時段           608
職務分類代碼           0
是否積極徵才           0
條件要求             0
福利制度             0
員工人數             0
搜尋關鍵字            0
dtype: int64

### 工作時段太多Null值，刪除該欄位

In [453]:
df.drop(columns=['工作時段','工作型態','職務分類代碼','薪資上限','福利制度'],inplace=True)

In [454]:
df['薪資是否為面議']= np.where(df['薪資下限']==0,1,0) #增加'薪資是否為面議'欄位
df['薪資下限']=df['薪資下限'].replace(0,40000)    #將面議的改成法規規定最低40000
df=df[df['薪資下限']>=1000]     #將時薪(非正職、學生實習)的資料去除

#自動判斷月薪超過300000為年薪，並除以12，轉為月薪
df['薪資下限'] = df['薪資下限'].astype(float)   #原本欄位是(int)類型，除12後會變float，會相衝，故先轉成float。
df.loc[df['薪資下限'] > 300000, '薪資下限']/= 12
df['薪資下限'] = df['薪資下限'].round(0).astype(int)


In [455]:
df.shape

(848, 19)

In [456]:
df['major']

0                                        []
1                                        []
2      ['數理統計相關', '工業工程相關', '其他數學及電算機科學相關']
3                                        []
4            ['一般商業學類', '醫藥工程相關', '數理統計相關']
                       ...                 
861          ['統計學相關', '資訊管理相關', '大眾傳播學相關']
862                                      []
863                                      []
864           ['統計學相關', '資訊工程相關', '資訊管理相關']
865             ['商業及管理學科類', '數學及電算機科學學科類']
Name: major, Length: 848, dtype: object

In [457]:
import ast # 導入 ast 模組
df["是否要求特定major"] = df['major'].apply(lambda x: 1 if len(ast.literal_eval(x)) > 0 else 0) #增加一欄，[是否要求特定major]的欄位。

In [458]:
df['是否要求特定major'].tail(20)

846    0
847    1
848    0
849    0
850    1
851    0
852    1
853    0
854    0
855    0
856    1
857    1
858    0
859    0
860    1
861    1
862    0
863    0
864    1
865    1
Name: 是否要求特定major, dtype: int64

In [459]:
df['是否遠距'].value_counts()

是否遠距
0    768
2     70
1     10
Name: count, dtype: int64

In [460]:
df['是否遠距'] = df['是否遠距'].astype(int)

In [461]:
remote_work={0:"無遠距",1:"完全遠距",2:"混和"}
df['是否遠距']=df['是否遠距'].map(remote_work)

In [462]:
df.head()

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,搜尋關鍵字,薪資是否為面議,是否要求特定major
0,20250709,5,雜誌／期刊出版業,今周刊_今周文化事業股份有限公司,1.【必備】Google Analytics技能。 \n2.【必備】熟稔Excel(函數、樞...,台北市,數據分析師【今周刊─數位內容部】,14090732,25.051911,121.526154,[],"[4, 5]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 64, 'descrip...",217人,數據分析,1,0
1,20250715,4,人力仲介代徵,藝珂人事顧問股份有限公司,數據洞察與分析\n分析商品分類、顧客行為、流量來源、轉換率等電商關鍵指標，找出成長機會與潛在...,台北市,[電商] 數據分析主管,14739101,25.040652,121.565297,[],[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,數據分析,1,0
2,20250717,6,半導體製造業,錼創顯示科技股份有限公司,1. 分析生產數據與大數據建模，協助建立品質監測與異常偵測模型，提供優化建議。\n2. 參與...,苗栗縣,數據分析工程師,14535114,24.711336,120.913172,"['數理統計相關', '工業工程相關', '其他數學及電算機科學相關']",[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",500人,數據分析,1,1
3,20250604,5,大專校院教育事業,國立臺北科技大學,作，成果應用於企業與政府智慧城市專案。加入我們，您將成為城市科學的先鋒，與世界領先的團隊共創...,台北市,MITxTaipei Tech 都市數據分析師,13817654,25.041802,121.534887,[],"[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,數據分析,1,0
4,20250625,11,工商顧問服務業,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,14079349,25.053372,121.522852,"['一般商業學類', '醫藥工程相關', '數理統計相關']","[2, 3, 4, 5, 6]",無遠距,50000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",20人,數據分析,0,1


In [463]:
df[df["工作編號"].duplicated()]

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,搜尋關鍵字,薪資是否為面議,是否要求特定major
170,20250716,5,生化科技研發業,俊質生醫股份有限公司,1.\t協助部門進行數據分析、模型建立。\n2.\t進行數據資料統計、彙整及報表。\n3.\...,台南市,數據分析工程師(南科),14437293,23.097413,120.284034,"['電機電子工程相關', '資訊工程相關']",[4],無遠距,38000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",20人,數據工程,0,1
190,20250714,6,電子通訊／電腦週邊零售業,全國電子股份有限公司,擔任數據平台的資料處理及報表開發工作，透過BI工具重新改寫既有的系統報表及資料處理流程，以提...,新北市,總公司-IT部門/數據分析及報表開發工程師,14010560,25.068519,121.446064,[],"[4, 5]",無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",1700人,數據工程,1,0
200,20250717,6,半導體製造業,錼創顯示科技股份有限公司,1. 分析生產數據與大數據建模，協助建立品質監測與異常偵測模型，提供優化建議。\n2. 參與...,苗栗縣,數據分析工程師,14535114,24.711336,120.913172,"['數理統計相關', '工業工程相關', '其他數學及電算機科學相關']",[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",500人,數據工程,1,1
203,20250715,3,其他專業／科學及技術業,工研院 _財團法人工業技術研究院,工作內容\n模擬數據處理、資料庫分析與預測模型生成 \n1.機器學習與預測模型建立：有相關實...,新竹縣,工研院機械所-電動車數據分析工程師(D400),14491156,24.774192,121.047917,"['電機電子工程相關', '機械工程相關']","[5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",6000人,數據工程,1,1
204,20250715,8,印刷電路板製造業(PCB),台郡科技股份有限公司(總公司),1. 開發與維護數據分析相關的 Web 系統。協助智慧工廠推進、強化工程師資料探勘能力。\n...,高雄市,數據分析工程師,12937778,22.593423,120.428297,"['資訊工程相關', '其他數學及電算機科學相關']","[4, 5]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",2000人,數據工程,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,20250709,1,金融控股業,國泰金控_國泰金融控股股份有限公司,比起工作內容，你應該要先認識我們的團隊：\n我們是一個正在成長的資料科學團隊，在找有熱情、想...,台北市,"資料科學架構師 Data Architect -數據科技-(數數發中心, DDT)_I000...",14044950,25.024944,121.543378,[],"[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",800人,數據科學,1,0
856,20250510,4,半導體製造業,台灣積體電路製造股份有限公司(台積電),【本職缺僅接受台積電官方網站投遞】 \n請至台積電官方網站投遞個人履歷表，此職缺履歷登錄網址...,新竹縣,"AI & Data Analysis Engineer (Dept: AQSD, Div: ...",14106415,24.742791,120.999103,"['機械工程相關', '電機電子工程相關', '資訊工程相關']","[5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,數據科學,1,1
858,20250718,2,多媒體傳播相關業,杰德創意影音管理股份有限公司,【負責品牌】\n全球LGBTQ+娛樂領導平台 GagaOOLala https://www....,台北市,數據分析師,14540514,25.024347,121.548339,[],"[4, 5, 6]",無遠距,48000,1,"{'acceptRole': {'role': [{'code': 32, 'descrip...",暫不提供,數據分析,0,0
863,20250714,10,網際網路相關業,盛形通運有限公司,一、工作內容?\n業務數據需求分析：研究運送平台等同類型平臺模式，結合臺灣本地業務，梳理司機...,台北市,數據分析專員,14604483,25.077849,121.568373,[],"[3, 4, 5]",無遠距,38000,0,"{'acceptRole': {'role': [{'code': 65536, 'desc...",18人,數據分析,0,0


In [464]:
df[df["工作編號"].isin([14720195,13634345])]

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,搜尋關鍵字,薪資是否為面議,是否要求特定major
188,20250716,22,家用電器製造業,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,14720195,25.068090,121.455088,"['數理統計相關', '其他數學及電算機科學相關', '工業工程相關']",[4],無遠距,38000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",108人,數據工程,0,1
608,20250715,19,光電產業,達興材料股份有限公司,1. 執行AI技術開發專案。\n2. 大數據分析與建模。\n3. 舉辦內部統計與數值分析的教...,台中市,研發-資料科學研究員,13634345,24.204932,120.607608,"['資訊工程相關', '數理統計相關', '其他數學及電算機科學相關']","[5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",430人,數據科學,1,1


In [465]:
df.drop_duplicates(subset="工作編號",keep="first",inplace=True)  #保留第一個

In [466]:
df.set_index("工作編號")

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,搜尋關鍵字,薪資是否為面議,是否要求特定major
工作編號,,,,,,,,,,,,,,,,,,,
14090732,20250709,5,雜誌／期刊出版業,今周刊_今周文化事業股份有限公司,1.【必備】Google Analytics技能。 \n2.【必備】熟稔Excel(函數、樞...,台北市,數據分析師【今周刊─數位內容部】,25.051911,121.526154,[],"[4, 5]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 64, 'descrip...",217人,數據分析,1,0
14739101,20250715,4,人力仲介代徵,藝珂人事顧問股份有限公司,數據洞察與分析\n分析商品分類、顧客行為、流量來源、轉換率等電商關鍵指標，找出成長機會與潛在...,台北市,[電商] 數據分析主管,25.040652,121.565297,[],[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,數據分析,1,0
14535114,20250717,6,半導體製造業,錼創顯示科技股份有限公司,1. 分析生產數據與大數據建模，協助建立品質監測與異常偵測模型，提供優化建議。\n2. 參與...,苗栗縣,數據分析工程師,24.711336,120.913172,"['數理統計相關', '工業工程相關', '其他數學及電算機科學相關']",[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",500人,數據分析,1,1
13817654,20250604,5,大專校院教育事業,國立臺北科技大學,作，成果應用於企業與政府智慧城市專案。加入我們，您將成為城市科學的先鋒，與世界領先的團隊共創...,台北市,MITxTaipei Tech 都市數據分析師,25.041802,121.534887,[],"[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,數據分析,1,0
14079349,20250625,11,工商顧問服務業,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,25.053372,121.522852,"['一般商業學類', '醫藥工程相關', '數理統計相關']","[2, 3, 4, 5, 6]",無遠距,50000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",20人,數據分析,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14325959,20250714,6,電腦軟體服務業,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,25.076077,121.575436,[],"[3, 4]",無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",20人,數據分析,1,0
14424533,20250715,5,廣告行銷公關業,華權數位整合行銷有限公司,主要工作述敘：\n1. 數據分析與精算：\n■分析遊戲玩家行為數據、直播主表現數據，協助優化...,台中市,數據分析師,24.116516,120.615540,"['統計學相關', '數理統計相關', '應用數學相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,數據分析,0,1
14724608,20250714,3,廣告行銷公關業,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\n\n2.消費者洞察: 掌握潛...,台北市,數據分析經理,25.034500,121.560053,"['統計學相關', '資訊管理相關', '大眾傳播學相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,數據分析,1,1


In [467]:
from cleaning_industry import clean_industry_column
df=clean_industry_column(df)
df.drop(columns=["產業"],inplace=True)

In [468]:
df["八大產業"].value_counts()

八大產業
資訊科技業       222
金融與保險業      110
服務與專業顧問業     93
製造業          90
批發與零售業       33
醫療與健康照護      20
其他           13
觀光與餐飲業        5
運輸與物流業        4
Name: count, dtype: int64

In [469]:
df.head()

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,搜尋關鍵字,薪資是否為面議,是否要求特定major,八大產業
0,20250709,5,今周刊_今周文化事業股份有限公司,1.【必備】Google Analytics技能。 \n2.【必備】熟稔Excel(函數、樞...,台北市,數據分析師【今周刊─數位內容部】,14090732,25.051911,121.526154,[],"[4, 5]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 64, 'descrip...",217人,數據分析,1,0,服務與專業顧問業
1,20250715,4,藝珂人事顧問股份有限公司,數據洞察與分析\n分析商品分類、顧客行為、流量來源、轉換率等電商關鍵指標，找出成長機會與潛在...,台北市,[電商] 數據分析主管,14739101,25.040652,121.565297,[],[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,數據分析,1,0,服務與專業顧問業
2,20250717,6,錼創顯示科技股份有限公司,1. 分析生產數據與大數據建模，協助建立品質監測與異常偵測模型，提供優化建議。\n2. 參與...,苗栗縣,數據分析工程師,14535114,24.711336,120.913172,"['數理統計相關', '工業工程相關', '其他數學及電算機科學相關']",[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",500人,數據分析,1,1,製造業
3,20250604,5,國立臺北科技大學,作，成果應用於企業與政府智慧城市專案。加入我們，您將成為城市科學的先鋒，與世界領先的團隊共創...,台北市,MITxTaipei Tech 都市數據分析師,13817654,25.041802,121.534887,[],"[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,數據分析,1,0,服務與專業顧問業
4,20250625,11,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,14079349,25.053372,121.522852,"['一般商業學類', '醫藥工程相關', '數理統計相關']","[2, 3, 4, 5, 6]",無遠距,50000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",20人,數據分析,0,1,服務與專業顧問業


In [470]:
df

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,搜尋關鍵字,薪資是否為面議,是否要求特定major,八大產業
0,20250709,5,今周刊_今周文化事業股份有限公司,1.【必備】Google Analytics技能。 \n2.【必備】熟稔Excel(函數、樞...,台北市,數據分析師【今周刊─數位內容部】,14090732,25.051911,121.526154,[],"[4, 5]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 64, 'descrip...",217人,數據分析,1,0,服務與專業顧問業
1,20250715,4,藝珂人事顧問股份有限公司,數據洞察與分析\n分析商品分類、顧客行為、流量來源、轉換率等電商關鍵指標，找出成長機會與潛在...,台北市,[電商] 數據分析主管,14739101,25.040652,121.565297,[],[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,數據分析,1,0,服務與專業顧問業
2,20250717,6,錼創顯示科技股份有限公司,1. 分析生產數據與大數據建模，協助建立品質監測與異常偵測模型，提供優化建議。\n2. 參與...,苗栗縣,數據分析工程師,14535114,24.711336,120.913172,"['數理統計相關', '工業工程相關', '其他數學及電算機科學相關']",[5],無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",500人,數據分析,1,1,製造業
3,20250604,5,國立臺北科技大學,作，成果應用於企業與政府智慧城市專案。加入我們，您將成為城市科學的先鋒，與世界領先的團隊共創...,台北市,MITxTaipei Tech 都市數據分析師,13817654,25.041802,121.534887,[],"[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,數據分析,1,0,服務與專業顧問業
4,20250625,11,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,14079349,25.053372,121.522852,"['一般商業學類', '醫藥工程相關', '數理統計相關']","[2, 3, 4, 5, 6]",無遠距,50000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",20人,數據分析,0,1,服務與專業顧問業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,20250714,6,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,[],"[3, 4]",無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",20人,數據分析,1,0,資訊科技業
860,20250715,5,華權數位整合行銷有限公司,主要工作述敘：\n1. 數據分析與精算：\n■分析遊戲玩家行為數據、直播主表現數據，協助優化...,台中市,數據分析師,14424533,24.116516,120.615540,"['統計學相關', '數理統計相關', '應用數學相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,數據分析,0,1,服務與專業顧問業
861,20250714,3,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\n\n2.消費者洞察: 掌握潛...,台北市,數據分析經理,14724608,25.034500,121.560053,"['統計學相關', '資訊管理相關', '大眾傳播學相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,數據分析,1,1,服務與專業顧問業
862,20250709,2,CONTIN 康定_奇來生技有限公司,團媽 合作效益追蹤機制，量化其曝光、導流與銷售成效，並提出合作優化建議。\n\n3.利用 G...,高雄市,數位行銷數據分析師,14620764,22.641587,120.385938,[],"[3, 4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,數據分析,0,0,製造業


In [471]:
df["學歷要求"].value_counts()

學歷要求
[4, 5, 6]             214
[4, 5]                118
[1, 2, 3, 4, 5, 6]     55
[3, 4, 5, 6]           53
[5, 6]                 48
[3, 4, 5]              30
[5]                    29
[4]                    16
[2, 3, 4, 5, 6]         9
[3, 4]                  9
[3]                     4
[2, 3, 4, 5]            2
[6]                     2
[2, 3, 4]               1
Name: count, dtype: int64

In [472]:
edu_map = {
            1: '不拘',
            2: '高中',
            3: '專科',
            4: '大學',
            5: '碩士',
            6: '博士',
        }
df['學歷要求']=df['學歷要求'].apply(lambda x:edu_map.get(min(ast.literal_eval(x))))

In [473]:
df['員工人數'] = df['員工人數'].apply(lambda x: int(x.replace('人', '')) if isinstance(x, str) and x.endswith('人') else pd.NA) # 去'人'轉數字，並將'暫不提供'改為NA

In [474]:
#增加[公司規模]欄位，依員工人數劃分
def classify_company_size(num):       
    if pd.isnull(num):
        return '未知'
    elif num<50:
        return '小型'
    elif num < 200:
        return '中型'
    else:
        return '大型'
df['公司規模']=df['員工人數'].apply(classify_company_size)

In [475]:
df['是否遠距'].value_counts()

是否遠距
無遠距     534
混和       51
完全遠距      5
Name: count, dtype: int64

In [476]:
def parse_string(x):
    """
    將字串轉換為 Python 的資料結構（如字典、列表、數字等）。
    """
    if isinstance(x,str):
        try:
            return ast.literal_eval(x)
        except Exception as e:
            return {}
    return x
df['條件要求']=df['條件要求'].apply(parse_string)   
#提取工作經驗
df['工作經驗']=df['條件要求'].apply(lambda x:x.get('workExp') if isinstance(x,dict) else None)
#提取專長工具
def format_specialty(job_require_dict):

    if isinstance(job_require_dict,dict) and 'specialty' in job_require_dict:
        specialty_list=job_require_dict['specialty']
        if isinstance(specialty_list,list):
            descriptions=[s.get('description') for s in specialty_list if isinstance(s,dict) and s.get('description')]
            return ", ".join(descriptions)

    return None


df['專長工具']=df['條件要求'].apply(format_specialty)
df.drop(columns='條件要求',inplace=True)

In [477]:
df.tail()

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,...,薪資下限,是否積極徵才,員工人數,搜尋關鍵字,薪資是否為面議,是否要求特定major,八大產業,公司規模,工作經驗,專長工具
859,20250714,6,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,[],...,40000,1,20,數據分析,1,0,資訊科技業,小型,1年以上,"Python, MySQL, PostgreSQL"
860,20250715,5,華權數位整合行銷有限公司,主要工作述敘：\n1. 數據分析與精算：\n■分析遊戲玩家行為數據、直播主表現數據，協助優化...,台中市,數據分析師,14424533,24.116516,120.615540,"['統計學相關', '數理統計相關', '應用數學相關']",...,40000,0,<NA>,數據分析,0,1,服務與專業顧問業,未知,1年以上,
861,20250714,3,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\n\n2.消費者洞察: 掌握潛...,台北市,數據分析經理,14724608,25.034500,121.560053,"['統計學相關', '資訊管理相關', '大眾傳播學相關']",...,40000,0,<NA>,數據分析,1,1,服務與專業顧問業,未知,3年以上,"Excel, Google Analytics"
862,20250709,2,CONTIN 康定_奇來生技有限公司,團媽 合作效益追蹤機制，量化其曝光、導流與銷售成效，並提出合作優化建議。\n\n3.利用 G...,高雄市,數位行銷數據分析師,14620764,22.641587,120.385938,[],...,40000,0,<NA>,數據分析,0,0,製造業,未知,2年以上,"Line, Instagram, Facebook, Google Analytics, G..."
864,20250718,0,喬山健康科技股份有限公司,1. 收集並整理各類數據，確保數據準確性和完整性；\n2. 運用統計學和數據視覺化工具，對數...,台北市,總部數位-數據分析師(台北中正區),14745746,25.044908,121.515143,"['統計學相關', '資訊工程相關', '資訊管理相關']",...,40000,1,8000,數據分析,1,1,其他,大型,7年以上,"Tableau, Power BI​, Looker studio, Python, MySQL"


In [478]:
df.shape

(590, 22)

In [ ]:
#暫時匯出
#desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "暫時匯出查看.csv")
#df.to_csv(desktop_path, index=False, encoding='utf-8-sig')